In [1]:
#Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


import missingno as msno
import pandas_profiling as pp
from pathlib import Path

%matplotlib inline
%precision %.2f

plt.style.use('bmh')
pd.set_option('display.max_rows', 30)
pd.set_option('display.min_rows', 10)
pd.set_option('display.max_columns', 100)

In [2]:
application = pd.read_csv('../data/raw/application_train.csv')
prev_application = pd.read_csv('../data/raw/previous_application.csv')
pos_cash_balance = pd.read_csv('../data/raw/POS_CASH_balance.csv')
credit_card_balance = pd.read_csv('../data/raw/credit_card_balance.csv')
installments_payments = pd.read_csv('../data/raw/installments_payments.csv')


In [4]:
application.columns = [column.lower() for column in application.columns]
prev_application.columns = [column.lower() for column in prev_application.columns]
pos_cash_balance.columns = [column.lower() for column in pos_cash_balance.columns]
credit_card_balance.columns = [column.lower() for column in credit_card_balance.columns]
installments_payments.columns = [column.lower() for column in installments_payments.columns]


In [51]:
sk_id_curr_cnt = prev_application.groupby('sk_id_curr').sk_id_prev.agg('count')
sk_id_curr_cnt_2 = sk_id_curr_cnt[sk_id_curr_cnt==2]
sk_id_curr_cnt_2

sk_id_curr
100005    2
100014    2
100017    2
100020    2
100034    2
         ..
456245    2
456246    2
456249    2
456253    2
456254    2
Name: sk_id_prev, Length: 52737, dtype: int64

In [16]:
instal = installments_payments.assign(
            # days
            payment_delay = (installments_payments.days_entry_payment - installments_payments.days_instalment),
            payment_advance = (installments_payments.days_instalment - installments_payments.days_entry_payment),
            # amounts
            payment_deficit = (installments_payments.amt_instalment - installments_payments.amt_payment),
            payment_surplus = (installments_payments.amt_payment - installments_payments.amt_instalment)
            )

instal.payment_delay.where(instal.payment_delay >=0,0,inplace=True)
instal.payment_advance.where(instal.payment_advance >=0,0,inplace=True)
instal.payment_deficit.where(instal.payment_deficit >= 0,0,inplace=True)
instal.payment_surplus.where(instal.payment_surplus >= 0,0,inplace=True)

In [43]:
instal_delayed = instal.loc[(instal.payment_delay > 0)]
instal_delayed

,sk_id_prev,sk_id_curr,num_instalment_version,num_instalment_number,days_instalment,days_entry_payment,amt_instalment,amt_payment,payment_delay,payment_advance,payment_deficit,payment_surplus
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,17.0,0.0,4.455,0.0
19,2329072,183463,1.0,11,-1699.0,-1693.0,7283.295,7283.295,6.0,0.0,0.000,0.0
30,1840786,175114,1.0,11,-476.0,-474.0,21179.700,21161.700,2.0,0.0,18.000,0.0
45,1644499,111587,1.0,8,-2116.0,-2101.0,3078.810,3078.810,15.0,0.0,0.000,0.0
47,1895324,131465,1.0,6,-1386.0,-1379.0,3229.065,3212.865,7.0,0.0,16.200,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13605308,1493367,415806,1.0,18,-587.0,-586.0,28873.170,28873.170,1.0,0.0,0.000,0.0
13605319,2003162,450999,1.0,16,-2135.0,-2113.0,5414.265,5384.880,22.0,0.0,29.385,0.0
13605328,2608958,408941,1.0,18,-41.0,-40.0,22788.135,22788.135,1.0,0.0,0.000,0.0
13605340,1635645,417079,1.0,1,-2221.0,-2215.0,9914.850,9010.485,6.0,0.0,904.365,0.0


In [68]:
both = instal_delayed.loc[instal_delayed.sk_id_curr.isin(sk_id_curr_cnt_2.index)]
both.head(20)

,sk_id_prev,sk_id_curr,num_instalment_version,num_instalment_number,days_instalment,days_entry_payment,amt_instalment,amt_payment,payment_delay,payment_advance,payment_deficit,payment_surplus
68,1780053,189947,1.0,5,-374.0,-373.0,22772.340,22772.340,1.0,0.0,0.000,0.0
153,1621918,197902,1.0,18,-21.0,-17.0,23772.105,23616.540,4.0,0.0,155.565,0.0
395,1307918,131426,1.0,5,-373.0,-364.0,10481.040,10195.200,9.0,0.0,285.840,0.0
468,1369281,107753,1.0,9,-1701.0,-1687.0,5626.710,5406.390,14.0,0.0,220.320,0.0
511,2673507,158970,1.0,5,-2372.0,-2344.0,15710.625,541.215,28.0,0.0,15169.410,0.0
1018,2642072,189820,1.0,10,-863.0,-861.0,16423.650,16423.650,2.0,0.0,0.000,0.0
1132,2355285,162162,1.0,11,-1454.0,-1447.0,6151.005,6151.005,7.0,0.0,0.000,0.0
1223,2070304,148695,1.0,14,-1965.0,-1954.0,9901.395,9861.030,11.0,0.0,40.365,0.0
1224,2051230,157499,0.0,66,-1886.0,-1877.0,8100.000,8100.000,9.0,0.0,0.000,0.0
1394,2315759,102017,0.0,20,-2637.0,-2633.0,9000.000,9000.000,4.0,0.0,0.000,0.0


In [69]:
sample_install = installments_payments.loc[installments_payments.sk_id_curr ==189820, 
                          ['sk_id_curr','sk_id_prev','days_instalment','days_entry_payment']]
sample_install.sort_values('sk_id_prev')
# 107753

,sk_id_curr,sk_id_prev,days_instalment,days_entry_payment
106492,189820,1873488,-2041.0,-2050.0
3339916,189820,1873488,-2071.0,-2085.0
2852662,189820,1873488,-2191.0,-2198.0
935070,189820,1873488,-2101.0,-2113.0
2057293,189820,1873488,-2131.0,-2143.0
1500848,189820,1873488,-2161.0,-2166.0
1018,189820,2642072,-863.0,-861.0
3043066,189820,2642072,-1133.0,-1147.0
2369272,189820,2642072,-833.0,-818.0
2104898,189820,2642072,-953.0,-953.0


In [74]:
sample_instal_fe = instal.loc[installments_payments.sk_id_curr ==189820, 
                          ['sk_id_curr','sk_id_prev','days_instalment','days_entry_payment','payment_delay','payment_deficit']]
sample_instal_fe.sort_values('sk_id_prev')


,sk_id_curr,sk_id_prev,days_instalment,days_entry_payment,payment_delay,payment_deficit
2852662,189820,1873488,-2191.0,-2198.0,0.0,0.0
1500848,189820,1873488,-2161.0,-2166.0,0.0,0.0
2057293,189820,1873488,-2131.0,-2143.0,0.0,0.0
935070,189820,1873488,-2101.0,-2113.0,0.0,0.0
3339916,189820,1873488,-2071.0,-2085.0,0.0,0.0
106492,189820,1873488,-2041.0,-2050.0,0.0,0.0
598459,189820,2642072,-803.0,-812.0,0.0,0.0
2369272,189820,2642072,-833.0,-818.0,15.0,0.0
1018,189820,2642072,-863.0,-861.0,2.0,0.0
1473166,189820,2642072,-893.0,-885.0,8.0,0.0


In [72]:
instal_col_sort = ['sk_id_curr','sk_id_prev','num_instalment_number']
instal.sort_values(by = instal_col_sort,inplace= True)

instal_grp = instal.groupby(['sk_id_curr','sk_id_prev'],as_index=False).agg(
        # days
        in_sum_payment_delay = pd.NamedAgg(column = 'payment_delay', aggfunc = 'sum'),
        in_sum_payment_advance = pd.NamedAgg(column = 'payment_advance', aggfunc = 'sum'),
        # counts
        in_cnt_payment = pd.NamedAgg(column = 'amt_payment', aggfunc = 'count'),
        in_cnt_version = pd.NamedAgg(column='num_instalment_version', aggfunc = 'nunique'),
        # amounts
        in_sum_payment_deficit = pd.NamedAgg(column = 'payment_deficit', aggfunc = 'sum'),
        in_sum_payment_surplus = pd.NamedAgg(column = 'payment_surplus', aggfunc = 'sum'),
        in_sum_instal = pd.NamedAgg(column = 'amt_instalment', aggfunc = 'sum'),
        in_sum_payment = pd.NamedAgg(column = 'amt_payment', aggfunc = 'sum')
            )

In [ ]:
instal_grp = instal_grp.assign(
        in_rt_cnt_deficit_pmt = instal_grp.in_cnt_payment_deficit/instal_grp.in_cnt_payment,
        in_rt_cnt_surplus_pmt = instal_grp.in_cnt_payment_surplus/instal_grp.in_cnt_payment,
        in_rt_amt_deficit_inst = instal_grp.in_sum_payment_deficit/instal_grp.in_sum_instal,
        in_rt_amt_surplus_inst = instal_grp.in_sum_payment_surplus/instal_grp.in_sum_instal,
        )
instal_grp.head()

In [76]:
sample_instal_grp = instal_grp.loc[instal_grp.sk_id_curr == 189820, 
                          ['sk_id_curr','sk_id_prev','in_sum_payment_delay']]
sample_instal_grp.sort_values('sk_id_prev')

,sk_id_curr,sk_id_prev,in_sum_payment_delay
250823,189820,1873488,0.0
250824,189820,2642072,34.0
